<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/Sharpe_Rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

샤프지수 = (포트폴리오 수익률 - 무위험금리)/포트폴리오 변동성<BR>
샤프지수를 함수로 작성하여, 주식과 주가지수 그룹에 대해 구해본다.<BR>
우선 필요한 모듈을 임포트 한다.

In [82]:
import yfinance as yf
import pandas as pd
import numpy as np

Sharpe 지수를 구하는 함수를 정의한다.<BR>
1. 인수는 symbol과 name, 시작일, 마지막일이고, name을 생략하면, symbol을 name으로 사용한다.<BR>
2. 반환값은 샤프지수를 가진 dataframe이며, dataframe열 이름은 name list를 사용<BR>

In [83]:
def GetSharpeRatio( tickers, names , start_date, end_date, rf=0 ):

  if not names: # names에 아무것도 안들어 오면 tickers를 names로 사용
    names = tickers

  # 샤프지수 계산에 필요한 dataframe들을 준비한다.
  df = pd.DataFrame()  # 가격
  aRet = pd.DataFrame()  # 수익률
  aStd = pd.DataFrame() # 표준편차
  Sharpe = pd.DataFrame() # 샤프지수

  # 목록에 있는 항목들을 하나씩 보면서 게산을 한다.
  for index, stock in enumerate(tickers):
    df[ names[index] ] = yf.download( stock, start=start_date, end=end_date , progress= False )["Adj Close"]
    aRet[names[index]] = df[names[index]].resample('Y').apply(lambda x: (x[-1] - x[0]) / x[0] -rf).ffill() # 연간 수익률 파악
    ntd = df[names[index]].resample('Y').count()    # 연간 거래일 수 파악

    # 일변동성을 측정한 다음, 이를 연간의 변동성으로 환산하기 위해 연간 거래일 수를 마지막에 곱한다.
    aStd[names[index]] = df[names[index]].resample('Y').apply(lambda x: x.pct_change().std())*np.sqrt(ntd)
    Sharpe[names[index]] = ( aRet[names[index]] - rf ) / aStd[names[index]]

  return Sharpe

주가를 읽어올 업체들의 symbol과 회사명을 리스트로 작성한다.

In [84]:
stocks = ['005930.KS', '066570.KS', '003490.KS', '010950.KS', '051910.KS', '017670.KS', '000100.KS', '000087.KS', '000070.KS', '000150.KS', '000270.KS' , '057050.KS' ] #symbol
names = ['Samsung', 'LGE', 'KAL', 'G-Cross', 'S-Oil', 'SK-Tel', 'Yoohan', 'HITE', 'Meritz', 'Doosan' , "KIA", "HD-HS"] # Name

주가를 불러올 기간을 설정한다.

In [85]:
start_date = "2015-1-31"
end_date = "2022-12-31"

무위험 금리를 설정한다. 연간 1%로 설정한다.

In [86]:
RiskFree = 0.01

In [ ]:
st_sharpe = GetSharpeRatio( stocks, names, start_date, end_date, RiskFree )

print( st_sharpe  )

샤프지수를 도식화 해 본다.

In [ ]:
st_sharpe.plot()

In [ ]:
st_sharpe.describe()

이번에는 5가지 주가지수를 가지고 샤프지수를 구해 본다.

In [ ]:
idx_tickers = [ "^KS11", "^GSPC" , "^DJI" , "^RUT" ,  "^HSI"]
names = [ "KOSPI", "S&P500", "Dow-Jones", "Rusell2000",  "Hang Seng"]

start_date = "2015-01-01"
end_date = "2022-12-31"

idx_sharpe = GetSharpeRatio( idx_tickers, names , start_date, end_date )

print( idx_sharpe )

샤프지수를 도식화 해본다.

In [ ]:
idx_sharpe.plot()

요약통계량을 출력해 본다.

In [ ]:
idx_sharpe.describe()